## Transfer Learning with VGG16 example

In [ ]:
# Local imports
import adjust_path  # Before doing any local imports
from icc.data_loader import DataLoader
from icc.models.finetuning_models import *

In [ ]:
X, y = DataLoader.load_train()

In [ ]:
# You must the bottleneck features first.
#tl = TransferLearnModel(basenet='VGG16', verbose=True, save_bottleneck_feats=True)

# Execute line only if you have saved bottleneck features.
tl = TransferLearnModel(basenet='VGG16', verbose=False, save_bottleneck_feats=False)
tl.fit(X, y)

In [ ]:
probabilities = tl.predict_proba(X)

In [ ]:
predictions = tl.predict(X)

### Extra fluff: nothing to see here.

In [ ]:
# Build VGG16 model and load weights
vgg_model = VGG16(input_shape=(75,75,3), include_top=False, weights='imagenet', classes=1, pooling=max)
vgg_input = Input(shape=(75,75,3), name='image_input')
vgg_output = vgg_model(vgg_input)

# Top portion of model
x_top = GlobalMaxPooling2D()(vgg_output)
x_top = Dense(1)(x_top)
x_top = Activation('sigmoid', name='predictions')(x_top)

# Create my own model
model = Model(inputs=vgg_input, outputs=x_top)

model.summary()

# Optimization algorithm for training model
opt = Nadam(lr=0.0001, epsilon=1e-8)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Fit model with data
model_history = model.fit(X_train, y_train, 
                          epochs=5,
                          verbose=2,
                          batch_size=32,
                          validation_data=(X_valid, y_valid))